In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import tensorflow as tf
print(tf.__version__)

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.33

1.5.0


In [3]:
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14361582112064692383
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14819619636
locality {
  bus_id: 1
}
incarnation: 4032390799241720485
physical_device_desc: "device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:09:00.0, compute capability: 7.0"
]


In [4]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
files, valid_files, test_files, N_V_trans, S_V_trans, X_V_trans, N_trans, S_trans, X_trans = [[] for i in range(9)]
files = os.listdir('/N/u/pmadinen/Assignment 2/tr')
valid_files = os.listdir('/N/u/pmadinen/Assignment 2/v')
files.sort()
valid_files.sort()

### Importing audio signals and trasformig them into spectrograms

In [6]:
for i in range(1200):
    s, sr=librosa.load('/N/u/pmadinen/Assignment 2/tr/' + files[i], sr=None)
    N_trans.append(np.transpose(np.abs(librosa.stft(s, n_fft=1024, hop_length=512))))

    s, sr=librosa.load('/N/u/pmadinen/Assignment 2/tr/' + files[i + 1200], sr=None)
    S_trans.append(np.transpose(np.abs(librosa.stft(s, n_fft=1024, hop_length=512))))
    
    s, sr=librosa.load('/N/u/pmadinen/Assignment 2/tr/' + files[i + 2400], sr=None)
    X_trans.append(np.transpose(np.abs(librosa.stft(s, n_fft=1024, hop_length=512))))

In [7]:
for i in range(1200):
    s, sr=librosa.load('/N/u/pmadinen/Assignment 2/v/' + valid_files[i], sr=None)
    N_V_trans.append(np.transpose(np.abs(librosa.stft(s, n_fft=1024, hop_length=512))))

    s, sr=librosa.load('/N/u/pmadinen/Assignment 2/v/' + valid_files[i + 1200], sr=None)
    S_V_trans.append(np.transpose(np.abs(librosa.stft(s, n_fft=1024, hop_length=512))))
    
    s, sr=librosa.load('/N/u/pmadinen/Assignment 2/v/' + valid_files[i + 2400], sr=None)
    X_V_trans.append(np.transpose(np.abs(librosa.stft(s, n_fft=1024, hop_length=512))))

In [8]:
max = X_trans[i].shape[0]
for i in range(len(X_trans)):
    if max <= X_trans[i].shape[0] :
        max = X_trans[i].shape[0]
print(max)

178


In [9]:
#Since the maximum number of row for a signal generated is 178, preparing the IBM matrices with order 178 * 513
M_trans = np.zeros((1200, 178, 513))
X_input = np.zeros((1200, 178, 513))

for i in range(len(X_trans)):
    for j in range(len(X_trans[i])):
        for k in range(len(X_trans[i][j])):
            X_input[i][j][k] = X_trans[i][j][k]
            if S_trans[i][j][k] > N_trans[i][j][k]:
                M_trans[i][j][k] = 1
            else:
                M_trans[i][j][k] = 0 

In [10]:
M_V_trans = np.zeros((1200, 178, 513))
X_V_input = np.zeros((1200, 178, 513))

for i in range(len(X_V_trans)):
    for j in range(len(X_V_trans[i])):
        for k in range(len(X_V_trans[i][j])):
            X_V_input[i][j][k] = X_V_trans[i][j][k]
            if S_V_trans[i][j][k] > N_V_trans[i][j][k]:
                M_V_trans[i][j][k] = 1
            else:
                M_V_trans[i][j][k] = 0

In [28]:
time_steps = 10
Neurons = 128
learning_rate = 0.001
batch_size = 64
hold_prob = 0.99

In [29]:
M_trans = M_trans.reshape(-1, time_steps, 513)
X_input = X_input.reshape(-1, time_steps, 513)

In [30]:
M_V_trans = M_V_trans.reshape(-1, time_steps, 513)
X_V_input = X_V_input.reshape(-1, time_steps, 513)

In [31]:
print(M_trans.shape, X_input.shape, M_V_trans.shape, X_V_input.shape)

(21360, 10, 513) (21360, 10, 513) (21360, 10, 513) (21360, 10, 513)


In [32]:
# Creating a function that returns random batches of rows for training

def Batching(x, y, batch_size, shuffle = False):
    if shuffle:
        indices = np.arange(x.shape[0])
        np.random.shuffle(indices)
    for idx in range(0, x.shape[0] - batch_size + 1, batch_size):
        if shuffle:
            batch = indices[idx: idx + batch_size]
        else:
            batch = slice(idx, idx + batch_size)
        yield x[batch], y[batch]

In [33]:
#tf.reset_default_graph()

In [34]:
# Defining the place holders

X = tf.placeholder(tf.float32, [None, None, 513])
y = tf.placeholder(tf.float32, [None, None, 513])

In [35]:
# Defining the RNN layer

RNN = tf.contrib.rnn.OutputProjectionWrapper(tf.contrib.rnn.BasicLSTMCell(num_units = Neurons, activation = tf.nn.sigmoid),
    output_size = 513)
RNN = tf.nn.rnn_cell.DropoutWrapper(RNN, output_keep_prob = hold_prob)

outputs, states = tf.nn.dynamic_rnn(RNN, X, dtype=tf.float32)

In [36]:
# Defining the loss

MSE = tf.reduce_mean(tf.pow(outputs - y, 2))

In [37]:
# Defining the Optimizer Function

optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
train = optimizer.minimize(MSE)

In [38]:
# Creating the Session

sess = tf.InteractiveSession(config = config)

In [39]:
# Creating the Session

init = tf.global_variables_initializer()
sess.run(init)

In [40]:
iterations = 30

for i in range(iterations + 1):

    for batch in Batching(X_input, M_trans, batch_size, shuffle = True):
         batch_X , batch_Y = batch
    
         sess.run(train, feed_dict={X: batch_X, y: batch_Y})

    if i% 6 == 0 :
        loss = sess.run(MSE, feed_dict={X: X_V_input, y: M_V_trans})

        print("Currently on Epoch # {}".format(i))
        print("MSE = {}".format(loss))
        print("\n")

Currently on Epoch # 0
MSE = 0.11759767681360245


Currently on Epoch # 6
MSE = 0.09419693797826767


Currently on Epoch # 12
MSE = 0.08864549547433853


Currently on Epoch # 18
MSE = 0.084555983543396


Currently on Epoch # 24
MSE = 0.08415083587169647


Currently on Epoch # 30
MSE = 0.08239462971687317




### Importing test signals, cleaning them and reconverting them back to auido signals

In [43]:
test_files = os.listdir('/N/u/pmadinen/Assignment 2/te')
test_files.sort()

In [57]:
for i in range(len(test_files)):
    s, sr = librosa.load('/N/u/pmadinen/Assignment 2/te/' + test_files[i], sr=None)
    test_complex = librosa.stft(s, n_fft=1024, hop_length=512)
    test = np.transpose(np.abs(test_complex))
    test = test.reshape(1, -1, 513)
    test_clean_trans = sess.run(outputs, feed_dict={X: test})
    test_clean_reshaped = test_clean_trans.reshape(-1, 513) 
    test_clean = np.transpose(test_clean_reshaped)
    test_phase = np.divide(test_complex, np.abs(test_complex))
    test_clean_phase = np.multiply(test_clean, test_phase)
    sh_T1 = librosa.istft(test_clean_phase,hop_length=512)
    librosa.output.write_wav('/N/u/pmadinen/Assignment 2/Clean Test Files/tex_s_recons_' + str(i) + '.wav', sh_T1, sr)
              

In [58]:
import zipfile

zf = zipfile.ZipFile("Clean Files.zip", "w")
for dirname, subdirs, files in os.walk("/N/u/pmadinen/Assignment 2/Clean Test Files"):
    zf.write(dirname)
    for filename in files:
        zf.write(os.path.join(dirname, filename))
zf.close()